In [1]:
# 도구 불러오기
import pandas as pd
import time # 실행에 대한 딜레이를 주기 위한 라이브러리
from selenium import webdriver as wb
from selenium.webdriver.common.by import By

In [2]:
# url 정의
url = 'https://www.hsd.co.kr/menu/menu_list'

# 크롬 브라우저 요청
driver = wb.Chrome()

driver.get(url)

1. '더보기'html요소 접근하기
2. '더보기'요소 클릭한다.(while)
3. 클릭을 완료했을 때 모든 메뉴정보가 브라우저에 출력되어야 한다.
4. 메뉴명, 가격 정보를 수집한다.
5. 메뉴명, 가격 정보를 텍스트 내용만 가져온 후 리스트에 저장한다.
6. 각 리스트를 이용하여 딕셔너리로 정리 -> DataFrame으로 생성한다

###  '더보기' html요소 접근하기

In [3]:
driver.find_element(By.CSS_SELECTOR, value = '.c_05').click()

### 
3. 클릭을 완료했을 때 모든 메뉴정보가 브라우저에 출력되어야 한다.

In [4]:
# 2~3 : 예외처리(try_except문법)
try :
    while True:
        driver.find_element(By.CSS_SELECTOR, value = '.c_05').click()
        # 클릭한 후 1초 뒤에 동작
        # -> 네트워크 속도에 따라서 컨텐츠가 보여지는 로딩 시간이 다름
        # sleep() : 딜레이 적용
        # 2 ~ 5초 사이로 설정
        # 활용 : 페이지 이동, 스크롤, 클릭
        time.sleep(1)
except:
    print('더 보기 클릭 완료!')

더 보기 클릭 완료!


### 메뉴명, 가격 정보를 수집한다

### 메뉴명, 가격 정보의 텍스트 내용만 가져온 후 리스트에 저장한다.

In [5]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
header = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

In [6]:

# html 요청
res = req.get(url , headers = header)
res
# 파이썬 객체 변환
html = bs(res.text, 'lxml') # 파싱 parsing(형식)
html

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no" name="viewport"/>
<!-- SNS LINK -->
<meta content="website" property="og:type"/>
<meta content="" property="og:title"/>
<meta content="" property="og:url"/>
<meta content="" property="og:image"/>
<meta content="" property="og:description"/>
<!--// SNS LINK -->
<meta content="_csrf" name="_csrf_parameter"/><meta content="X-CSRF-TOKEN" name="_csrf_header"/><meta content="434faad9-a935-43f9-9a71-e396ae844096" name="_csrf"/><title>전체메뉴 &gt; 한솥 메뉴</title>
<link href="/assets/images/common/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/assets/css/ui.common.css" rel="stylesheet" type="text/css"/>
<link href="/assets/css/swiper.min.css" rel="stylesheet" type="text/css"/>
<!--[if gte IE 9 ]><link rel="stylesheet" href

In [7]:
name = driver.find_elements(By.CSS_SELECTOR, value = '.h.fz_03')
price = driver.find_elements(By.CSS_SELECTOR, value = '.item-price>strong')

In [8]:
print(len(name),len(price))

91 91


In [9]:
rank_lst = []
name_lst_txt =[]
price_lst_txt = []

for i in range(len(name)):
    rank_lst.append(i+1)
    name_lst_txt.append(name[i].text)
    price_lst_txt.append(price[i].text)
print(len(rank_lst), len(name), len(price))

91 91 91


In [10]:
han_dict = {'' : rank_lst, 
              '메뉴이름' : name_lst_txt,
              '가격':  price_lst_txt }

data = pd.DataFrame(han_dict)
han_df2 = data.set_index('')
han_df2

,메뉴이름,가격
,,
1,[반찬] 사골 우거지 곰탕,"4,900"
2,[착한도시락] 참치야채 감초고추장 사골 우거지 곰탕 정식,"7,500"
3,[착한도시락] 빅치킨마요 사골 우거지 곰탕 정식,"7,900"
4,[착한도시락] 제육 사골 우거지 곰탕 정식,"8,900"
5,[착한도시락] 사골 우거지 곰탕,"6,500"
...,...,...
87,무말랭이 무침,400
88,한솥밥,"1,000"
89,현미밥,"1,700"


In [11]:
# 한솥 전체 메뉴 중에서 5000원 이하인 메뉴는 몇개 있을까?
# 그 메뉴들의 이름은 뭘까?
# 1. 콤마를 제거한 후 가격 컬럼 데이터 타입 변환
# 컬럼 인덱싱하는 방법
# df['컬럼명'], df.loc[:,'컬럼명'],df.iloc[:,컬럼인덱스]
han_df2.loc[:,'가격']
han_df2.iloc[:,1]


1     4,900
2     7,500
3     7,900
4     8,900
5     6,500
      ...  
87      400
88    1,000
89    1,700
90    1,500
91    2,200
Name: 가격, Length: 91, dtype: object

In [12]:
han_df2['가격']=han_df2['가격'].str.replace(',','').astype('int64')

,메뉴이름,가격
,,
1,[반찬] 사골 우거지 곰탕,4900
2,[착한도시락] 참치야채 감초고추장 사골 우거지 곰탕 정식,7500
3,[착한도시락] 빅치킨마요 사골 우거지 곰탕 정식,7900
4,[착한도시락] 제육 사골 우거지 곰탕 정식,8900
5,[착한도시락] 사골 우거지 곰탕,6500
...,...,...
87,무말랭이 무침,400
88,한솥밥,1000
89,현미밥,1700


In [18]:
han_len = han_df2[han_df2['가격'] <= 5000].shape[0]
han_len

52

In [21]:
han_len1 = han_df2[han_df2['가격'] <= 5000]['메뉴이름']
han_len1


1        [반찬] 사골 우거지 곰탕
22          칠리 찹쌀탕수육도련님
24                 치킨제육
25               돈까스도련님
26                 제육볶음
27             돈치스팸 도시락
36                 스팸마요
38                왕치킨마요
39                빅치킨마요
40                 치킨마요
41                 참치마요
42                 돈치마요
43               돈까스 카레
44             스팸 김치볶음밥
45                김치볶음밥
46              스팸철판볶음밥
47           소불고기 철판볶음밥
49             묵은지 김치찌개
52              마파두부 덮밥
54             새우돈까스 덮밥
55               돈까스 덮밥
57       시골제육 두부강된장 비빔밥
58           참치야채 감초고추장
59                튼튼도시락
62           토마토 미트 파스타
63              반찬 계란말이
64           반찬 묵은지김치찌개
65                반찬 카레
66           반찬 김치 부대찌개
67                반찬 치킨
68              반찬 고기고기
69           반찬 돈까스 도련님
70              반찬 제육볶음
71          반찬 토네이도 소세지
72             반찬 반달돈까스
73              뉴 감자고로케
74             미니 찹쌀핫도그
75             미니 찹쌀탕수육
76               케이준후라이
78    후라이드 순살(소)_양념치킨소스
79     후라이드 순살(소)_케이준소스
81          오리지